In [23]:
import requests
import yaml
import json
import gzip
import pandas as pd

In [24]:
import json
import os
import pandas as pd
import numpy as np
from pathlib import Path
import collections
from sklearn.model_selection import train_test_split
from sklearn import metrics

import sys
sys.path.append("../")
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

In [25]:
# payload = {'out': 'json', 'lim':2}
payload = {'out': 'json','gzip':1,'of':'n','lim':50}
res = requests.get('https://api.syosetu.com/novelapi/api/', params=payload).content
r =  gzip.decompress(res).decode("utf-8") 
allcount = json.loads(r)[0]["allcount"]
print(r)

[{"allcount":870190},{"ncode":"N5497GQ"},{"ncode":"N3115HN"},{"ncode":"N2740HN"},{"ncode":"N4650HA"},{"ncode":"N1172EV"},{"ncode":"N1863HN"},{"ncode":"N6751GT"},{"ncode":"N8571FQ"},{"ncode":"N4232FJ"},{"ncode":"N5984HI"},{"ncode":"N3114HN"},{"ncode":"N2899HN"},{"ncode":"N8290HM"},{"ncode":"N7729HM"},{"ncode":"N2228HN"},{"ncode":"N7544HJ"},{"ncode":"N0499HI"},{"ncode":"N2110HL"},{"ncode":"N0101HN"},{"ncode":"N8561HM"},{"ncode":"N2602HK"},{"ncode":"N2796HN"},{"ncode":"N9864HG"},{"ncode":"N3011HN"},{"ncode":"N5603HL"},{"ncode":"N8548HM"},{"ncode":"N3787GZ"},{"ncode":"N2840HD"},{"ncode":"N3113HN"},{"ncode":"N1577FQ"},{"ncode":"N6376HM"},{"ncode":"N9677GY"},{"ncode":"N9320HM"},{"ncode":"N2012HI"},{"ncode":"N4281HD"},{"ncode":"N2774HM"},{"ncode":"N2076HK"},{"ncode":"N8248GJ"},{"ncode":"N3014HM"},{"ncode":"N5058HA"},{"ncode":"N6667HM"},{"ncode":"N9916HC"},{"ncode":"N1724FH"},{"ncode":"N8960HM"},{"ncode":"N3729HM"},{"ncode":"N6354GU"},{"ncode":"N6869HD"},{"ncode":"N3111HN"},{"ncode":"N9017HL"}

In [26]:
allcount

870190

In [27]:
payload = {'out': 'json','gzip':5,'order':'weeklypoint','lim':500}
res = requests.get("https://api.syosetu.com/novelapi/api/?genre=102&length=1000-&type=r", params=payload, timeout=30).content
r =  gzip.decompress(res).decode("utf-8")
df_temp = pd.read_json(r)

In [28]:
df_temp
df_temp = df_temp.drop(0)

In [29]:
ncodes = list(df_temp.ncode.values)

In [30]:
from urllib import request
from bs4 import BeautifulSoup
import re


In [31]:
def get_one_page(url):
    # url = "http://www.yomiuri.co.jp/"
    text_list = []
    try:
        html = request.urlopen(url)

        #set BueatifulSoup
        soup = BeautifulSoup(html, "html.parser")

        #get 本文
        contents = soup.find("div", id="novel_honbun", class_="novel_view")
        contents_p = contents.find_all("p")
        
        for text in contents_p:
            # line = re.sub("<.+>", "", str(line))
            text = text.get_text().strip()
            if not text :
                continue
            text_list.append(text)

        return text_list
        
    except Exception as E:
        return text_list
    

In [32]:
base_url = "https://ncode.syosetu.com/"
def crawl_one_novel(ncode:str, start=1, end=10):
    novel_url = base_url+ncode.lower()+"/"
    novel = []
    for i in tqdm( range(start, end+1)) :
        url = novel_url+str(i)
        page = get_one_page(url)
        novel += page
    return novel

In [38]:
def clean_text_plain(text):
    text_ = neologdn.normalize(text)
    text_ = re.sub(r'\(.*\)', "", text_)
    text_ = re.sub(r'\d+', "0", text_)
    if "……" in text_:
        text_ = text_.replace("……", "…")
    return text_

In [34]:
def format_EOS(sentence):
    if sentence[-1] in ["。", "?", "!", "．"]:
        return sentence
    else:
         return sentence+"。"

In [35]:
def is_valid_utterance_segment(text:str)->bool:
    # 複数の"「"には囲まれていない
    if text.count("「") == 1 and text.count("」") == 1:
        return True
    return False

In [39]:
pattern = '(?<=\「).*(?=\」)'
for ncode in ncodes:
    print(ncode)
    novel = crawl_one_novel(ncode, end=10)
    convs = []
    for text in tqdm(novel):
        if is_valid_utterance_segment(text):
            utt = re.search(pattern, text).group()
            utt = clean_text_plain(utt)
            utt = format_EOS( utt )
            convs.append(utt)
    # print(convs)
    break

 10%|█         | 1/10 [00:00<00:01,  7.22it/s]

N8033HM


100%|██████████| 845/845 [00:00<00:00, 16198.52it/s]

ねぇねぇオタク君
えっ、オタク君って僕の事?
そうだよ。なんで?
苗字が小田倉なだけで、オタクじゃないよ
でもオタク君、授業中に女の子の絵描いてたりするじゃん?
なっ!?
そ、それで用事は何かな?
それがさ、このネイル。ヤバくね?
ヤバくね?
オタク君
えっと、どうヤバいのかな?
これ自分で塗ってみたんだけど、どうよ?
なんというか、独特な模様だね
あー、やっぱりか
実はさ、これの真似したんだけど上手くいかなくてさ
こんな感じにしようと思ったんだけどさ、中々上手くいかないんだよね
ふむ
確かにヤバいですね
でしょ!ヤバイよね!
そうだ。オタク君手先器用でしょ?これ作ってよ!
ちょっ、いくら絵が描けるからって、こんなのは…
あー…やっぱ無理だよね。ごめんね無理言っちゃって
出来るよ
えっ?
これくらいなら、出来るかなー
マジで!オタク君やばくね!
そ、そうかな
ねぇオタク君、お願い。これと同じの作って
良いけど、その代わり僕が絵を描いてた事とか誰かに喋ったりしないなら
本当に!うんうん、約束する!オタク君ありがとう!
えっ、ちょっと何してるの!?
ん?爪剥がしてるんだけど?
えっと、この付け爪に、さっきの画像のネイルをすれば良いんだよね?
そうそう。あっ、そうだ。オタク君が柄を忘れないように画像送るからライン教えてよ
あまりやった事ないけど、登録はこれで良いのかな?
オッケー。それじゃあ後で画像送るからよろしくね!
おーい。優愛なにしてるの?さっさと帰ろうよ
うん、今行く。それじゃあオタク君、バイバイ
はい。さよなら
それじゃあ、帰って作るかな
さてと、どうしたものか
とりあえず、塗料を剥がしておくか
除光液、0円もするのか…
シンナーでもいけるのか、それならあるぞ!
さてと、どうしたものか
本人に確認してみるかな
なんか固いな。クラスメイトなんだから、もうちょっとフレンドリーにした方が良いかな?
これだと馴れ馴れしい奴だな。友達面すんなとか言われそうだ
うおぉ
どうしよう。既読着いちゃってるよな
あっ、あっ
あっ、あっ
えっ
あっ、良いんだ
期間か、そうだな
集中すれば日付が変わる前には終わるかな
あら、こうちゃん。もう学校に行くの?
うん。今日は日直だから早いんだ
あら、そうなの。気を付けるのよ
それじゃあ、行ってきます
ふぁああ。眠い
あっ!オタク君おはよう!


In [40]:
novel

['「ねぇねぇオタク君」',
 'どこにでも居る、金髪ギャルの鳴海（なるみ）\u3000優愛（ゆあ）高校１年生。',
 '彼女がクラスメイトである小田倉（おたくら）\u3000浩一（こういち）に話しかけたのは、ただの気まぐれだった。',
 '放課後にいつも話す友達が居なかったので、暇つぶしにと、近くの席に居た小田倉に声をかけただけだった。',
 '「えっ、オタク君って僕の事？」',
 '「そうだよ。なんで？」',
 '小田倉は返答に困った。',
 '彼は確かにオタクではある。',
 'だが、かつて中学時代はそれで馬鹿にされたこともあり、高校になってからは極力周りにバレないようにしてきたつもりだ。',
 'だと言うのに、いきなりクラスメイトのギャルにオタク君呼ばわりされたのだ。',
 '（オタク呼ばわりは酷いけど、オタク君って呼ばれるのは嬉しいかも）',
 '彼はちょっとだけ、喜んだ。',
 '見事にツ〇ッター漫画に毒されているオタクである。',
 '「苗字が小田倉なだけで、オタクじゃないよ」',
 '「でもオタク君、授業中に女の子の絵描いてたりするじゃん？」',
 '「なっ！？」',
 '小田倉ことオタク君は気づいていなかった。',
 '自分の前や横の席から見えないようにしていても、自分の斜め後ろの席からは丸見えだったという事が。',
 '幸いにして彼の席は窓際の後ろから二番目。気づいて居るのは彼女だけだ。',
 '「そ、それで用事は何かな？」',
 '本人はさり気ないつもりだが、必死に話題を変えているのは一目瞭然だ。',
 '「それがさ、このネイル。ヤバくね？」',
 '優愛は彼の動揺など気にもせず、そう言って、手を出した。',
 '彼女にとって、オタク君が絵を描いている事など、どうでも良いからだ。',
 'オタク君がオタク趣味でも別に良くない？',
 'その程度の認識だ。',
 '全く興味のない爪を見せられた上に「ヤバくね？」と言われ、言葉を失うオタク君。',
 '人の事を「オタク君」と呼ぶ事の方がヤバくねと言いたい気持ちを、ぐっとこらえる。',
 '「えっと、どうヤバいのかな？」',
 '「これ自分で塗ってみたんだけど、どうよ？」',
 '優愛の爪を見るオタク君。',
 '爪にはカラフル、なんて形容詞では誤魔化せないような模様が出来ていた。',
 '絵画の授業

In [269]:
class Collector:
    def __init__(self, out_path="../../corpus/novel/"):
        self.out_path = out_path
        self.utt_set = set()
        self.title_set = set()
    
    def crawl(self, limit=5000):
        # limit 以下の場合に実行
        
        # 小説タイトルが沢山
        ncodes = self.get_titles()
        for i, ncode in enumerate(ncodes):
            # print(ncode)
            # タイトルが既に登録されていれば無視
            if ncode in self.title_set:
                continue
            # 会話文のみ
            convs = self.extract_convs_from_ncode(ncode, end=50)
            self.utt_set.update(set(convs))
                # 追加したタイトルを保存
            self.title_set.add(ncode)

            # 進捗
            if (i+1) % 10 == 0:
                print("complete rate => {0} : {1}%".format(len(self.utt_set), len(self.utt_set)*100//limit))
            
            # 上限を超えた！
            if len(self.utt_set) >= limit:
                break
        
        print("ended crawl/ crawled data => {0} : {1}%".format(len(self.utt_set), len(self.utt_set)*100//limit))
            
    
    def extract_convs_from_ncode(self, ncode, end=100) -> list:
        pattern = '(?<=\「).*(?=\」)'
        novel = crawl_one_novel(ncode, end=end)
        convs = []
        for text in novel:
            if "「" in text and "」" in text:  
                # print(text)
                utt = re.search(pattern, text).group()
                utt = clean_text_plain(utt)
                if utt == "":
                    continue
                # print(utt)
                utt = format_EOS( utt ) 
                convs.append(utt)
        return convs
    
    def get_titles(self):
        payload = {'out': 'json','gzip':5,'order':'yearlypoint','lim':500}
        res = requests.get("https://api.syosetu.com/novelapi/api/?genre=102&length=100000-&type=r", params=payload, timeout=30).content
        r =  gzip.decompress(res).decode("utf-8")
        df_temp = pd.read_json(r)
        df_temp = df_temp.drop(0)
        ncodes = list(df_temp.ncode.values)
        return ncodes
    
    def save(self, filename, titles):
        conv_data = dict()
        conv_data["utt"] = sorted( list(self.utt_set) ) 

        title_data = dict()
        title_data["ncode"] = sorted( list(self.title_set) ) 

        with open(self.out_path+filename, mode="w") as f:
            json.dump(conv_data, f, ensure_ascii=False, indent=4)
        
        with open(self.out_path+titles, mode="w") as f:
            json.dump(title_data, f, ensure_ascii=False, indent=4)
        
    def load(self, filename, titles):
        # 実データ
        if not os.path.exists(self.out_path+filename):
            print("file was not found : {0}".format(self.out_path+filename))
            return False
        else:
            # jsonファイル
            with open(self.out_path+filename, mode="r") as f:
                data = json.load(f)
            self.utt_set = set(data["utt"])
            print("success load : {0}".format(self.out_path+filename))
        
        # 読み込んだタイトル達
        if not os.path.exists(self.out_path+titles):
            print("file was not found : {0}".format(self.out_path+titles))
            return False

        else:
            with open(self.out_path+titles, mode="r") as f:
                data = json.load(f)
            self.ncode_set = set(data["ncode"])
            print("success load : {0}".format(self.out_path+titles))
        
        return True

In [270]:
collector = Collector()

In [271]:
filename = "novels.json"
titles = "ncodes.json"

In [272]:
collector.load(filename, titles)

file was not found : ../../corpus/novel/novels.json


False

In [274]:
collector.crawl(limit=500000)

  0%|          | 0/50 [00:00<?, ?it/s]

complete rate => 34636 : 6%


  2%|▏         | 1/50 [00:00<00:05,  9.36it/s]

complete rate => 47923 : 9%


  2%|▏         | 1/50 [00:00<00:05,  9.47it/s]

...

complete rate => 472672 : 94%


  2%|▏         | 1/50 [00:00<00:05,  9.23it/s]

complete rate => 483668 : 96%


100%|██████████| 50/50 [00:05<00:00,  9.06it/s]

complete rate => 500340 : 100%
ended crawl/ crawled data => 500340 : 100%


In [277]:
filename = "novels_{0}.json".format(50)
titles = "ncodes_{0}.json".format(50)

In [278]:
collector.save(filename, titles)